[참고](https://github.com/rickiepark/handson-ml2/blob/master/16_nlp_with_rnns_and_attention.ipynb)

- 챗봇(chatbot) : 대화의 상대방이 자신을 사람이라고 생각하도록 속일 수 있는 기계.
- 문자 단위 RNN(character RNN) : 문장에서 다음 글자를 예측하도록하는 훈련
- **상태가 없는 RNN**(stateless RNN)을 사용하고 **상태가 있는 RNN**(stateful RNN)을 구축한다.
- 텐서플로 애드온(Addon) 프로젝트에서 제공하는 seq2seq API를 사용한다.
- **트랜스포머**(transformer) : RNN을 모두 제거하고 어텐션만 사용해 매우 좋은 성능을 내는 구조. GPT-2와 BERT 같은 모델의 기반이 된다.

# 16.1 Char-RNN을 사용해 셰익스피어 문체 생성하기
## 16.1.1 훈련 데이터셋 만들기

In [2]:
# 셰익스피어 작품 다운로드
import tensorflow as tf
from tensorflow import keras

shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

1122304/1115394 [==============================] - 1s 1us/step


In [5]:
# 글자를 정수로 인코딩
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True) # 글자 수준 인코딩
tokenizer.fit_on_texts(shakespeare_text) 

In [6]:
tokenizer.texts_to_sequences(["First"])

[[20, 6, 9, 8, 3]]

In [7]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])

['f i r s t']

In [8]:
max_id = len(tokenizer.word_index) # number of distinct characters
dataset_size = tokenizer.document_count # total number of characters

In [10]:
# 전체 텍스트를 인코딩하여 각 글자를 ID로 나타냄
import numpy as np
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1 # 1에서 39 대신 0에서 38까지 ID를 얻기 위해 1을 뺌

## 16.1.2 순차 데이터셋을 나누는 방법
훈련 세트, 검증 세트, 테스트 세트가 중복되지 않도록 만드는 것이 중요. 시계열을 다룰 때는 보통 시간에 따라 나눈다. <br/>
암묵적으로 RNN은 시계열 데이터가 넓은 의미에서 **변하지 않는다**(stationary)고 가정한다.

In [11]:
# 텍스트 처음 90%를 훈련 세트로 사용
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

## 16.1.3 순차 데이터를 윈도 여러 개로 자르기
`window()` 메서드를 사용해 긴 시퀀스를 작은 많은 텍스트 윈도로 변환한다. <br/>
- TBPTT(truncated backpropagation through time) : 부분 문자열 길이만큼만 역전파를 위해 펼쳐지는 것.

In [12]:
n_steps = 100 # n_steps를 너무 짧게 하면 안 됨.
window_length = n_steps + 1 # target = input shifted 1 character ahead
dataset = dataset.repeat().window(window_length, shift=1, drop_remainder=True) # 모든 윈도가 동일한 글자를 포함하도록 지정.

- 중첩 데이터셋(nested dataset) : 하나의 데이터셋으로 표현되는 윈도를 포함하는 데이터셋. 섞거나 배치를 만들 때 유용.
- 플랫 데이터셋(flat dataset) : 데이터셋이 들어 있지 않는 데이터셋. 모델에 데이터셋이 아니라 텐서를 넣어야 하기 때문에 중첩 데이터셋 대신 사용.
- ex) {{1, 2}, {3, 4, 5}} (중첩 데이터셋) ---(`flat_map` 메서드)---> {1, 2, 3, 4, 5} (플랫 데이터셋)

In [13]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [14]:
# 윈도를 배치로 만들고 입력과 타깃을 분리
np.random.seed(42)
tf.random.set_seed(42)

batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [15]:
# 원-핫 벡터를 사용해 글자를 인코딩
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [16]:
# 프리페칭 추가
dataset = dataset.prefetch(1)

## 16.1.4 Char-RNN 모델 만들고 훈련하기

In [18]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     dropout=0.2), #recurrent_dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,
                     dropout=0.2), #recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, steps_per_epoch=train_size // batch_size,
                    epochs=3) # 엄청 오래 걸려서 3번만 돌림.

Epoch 1/3
31370/31370 [==============================] - 12929s 412ms/step - loss: 1.7195
Epoch 2/3
31370/31370 [==============================] - 4020s 128ms/step - loss: 1.5448
Epoch 3/3
31370/31370 [==============================] - 4000s 128ms/step - loss: 1.5209


## 16.1.5 Char-RNN 모델 사용하기

In [19]:
# 전처리를 위한 함수
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [20]:
# 어떤 텍스트의 다음 글자 예측
X_new = preprocess(["How are yo"])
Y_pred = model.predict_classes(X_new)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1] # 1st sentence, last char

C:\Users\sarah\anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


'u'

## 16.1.6 가짜 셰익스피어 텍스트를 생성하기
생성된 텍스트의 다양성을 제어하려면 **온도**(temperature)라고 불리는 숫자로 로짓을 나눈다. 0에 가까울수록 높은 확률을 가진 글자를 택한다.

In [21]:
tf.random.set_seed(42)

tf.random.categorical([[np.log(0.5), np.log(0.4), np.log(0.1)]], num_samples=40).numpy()

def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [22]:
# 함수를 반복 호출하여 다음 글자를 얻고 텍스트에 추가하는 함수
def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [23]:
# 온도를 다르게 해보며 테스트
tf.random.set_seed(42)

print(complete_text("t", temperature=0.2))

t the country have a suidor of the countrymen:
what


In [24]:
print(complete_text("t", temperature=1)) # 이 셰익스피어 모델은 1에 가까운 온도에서 잘 작동하는 듯.

thrick on my, my
high, thou art farcuus on toing yo


In [25]:
print(complete_text("t", temperature=2))

th 
reck padesic! ne'er fire-eat.
yot hrapicag spok


더 좋은 텍스트를 생성하려면 GRU 층과 뉴런 수를 늘리고 더 오래 훈련하거나 규제를 추가해보자!

## 16.1.7 상태가 있는 RNN
상태가 있는 RNN : 한 훈련 배치를 처리한 후에 마지막 상태를 다음 훈련 배치의 초기 상태로 사용하는 RNN <br/>
순차적이로 겹치지 않는 입력 시퀀스를 만들어야 하는데, `window()` 메서드에 `shift=n_steps`를 사용하면 된다.

In [26]:
tf.random.set_seed(42)

dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.repeat().batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [30]:
batch_size = 32
encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []
for encoded_part in encoded_parts:
    dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
    dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_length))
    datasets.append(dataset)
dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.repeat().map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

상태가 있는 RNN을 만들 때 주의할 점
1. 각 순환 층을 만들 때 `stateful=True`로 지정해야 함.
2. 배치 크기를 알아야 하기 때문에 첫번째 층에 `batch_input_shape` 매개변수를 지정한다.

In [31]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2, #recurrent_dropout=0.2,
                     batch_input_shape=[batch_size, None, max_id]),
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2), #recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

In [32]:
# 콜백 함수를 사용해 에포크 끝마다 상태를 재설정
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [33]:
# 모델을 컴파일하고 훈련
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
steps_per_epoch = train_size // batch_size // n_steps
history = model.fit(dataset, steps_per_epoch=steps_per_epoch, epochs=50,
                    callbacks=[ResetStatesCallback()])

Epoch 1/50
313/313 [==============================] - 38s 114ms/step - loss: 2.9076
Epoch 2/50
313/313 [==============================] - 36s 117ms/step - loss: 2.2875
Epoch 3/50
313/313 [==============================] - 37s 117ms/step - loss: 2.1358
Epoch 4/50
313/313 [==============================] - 35s 112ms/step - loss: 2.0491
Epoch 5/50
313/313 [==============================] - 35s 112ms/step - loss: 1.9944
Epoch 6/50
313/313 [==============================] - 37s 117ms/step - loss: 1.9538
Epoch 7/50
313/313 [==============================] - 35s 112ms/step - loss: 1.9246
Epoch 8/50
313/313 [==============================] - 36s 114ms/step - loss: 1.8992
Epoch 9/50
313/313 [==============================] - 35s 112ms/step - loss: 1.8817
Epoch 10/50
313/313 [==============================] - 35s 112ms/step - loss: 1.8652
Epoch 11/50
313/313 [==============================] - 35s 111ms/step - loss: 1.8542
Epoch 12/50
313/313 [==============================] - 36s 115ms/step - lo

# 16.2 감정 분석

In [34]:
# IMDb
tf.random.set_seed(42)

(X_train, y_test), (X_valid, y_test) = keras.datasets.imdb.load_data()
X_train[0][:10]

<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
C:\Users\sarah\anaconda3\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
C:\Users\sarah\anaconda3\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]

In [35]:
# 디코딩
word_index = keras.datasets.imdb.get_word_index()
id_to_word = {id_ + 3: word for word, id_ in word_index.items()}
for id_, token in enumerate(("<pad>", "<sos>", "<unk>")):
    id_to_word[id_] = token
" ".join([id_to_word[id_] for id_ in X_train[0][:10]])

'<sos> this film was just brilliant casting location scenery story'

In [36]:
# 텐서플로 데이터셋을 사용해 IMDb 리뷰를 텍스트로 적재
import tensorflow_datasets as tfds

datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_size = info.splits["train"].num_examples
test_size = info.splits["test"].num_examples

Dataset imdb_reviews downloaded and prepared to C:\Users\sarah\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


In [37]:
# 전처리 함수
def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300)
    X_batch = tf.strings.regex_replace(X_batch, rb"<br\s*/?>", b" ") # 줄바꿈을 공백으로 바꿈
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
    X_batch = tf.strings.split(X_batch)
    return X_batch.to_tensor(default_value=b"<pad>"), y_batch

In [38]:
# 어휘 사전 구축
from collections import Counter

vocabulary = Counter()
for X_batch, y_batch in datasets["train"].batch(32).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))

In [39]:
# 가장 많이 등장하는 단어 세 개 (???)
vocabulary.most_common()[:3]

[(b'<pad>', 214309), (b'the', 61137), (b'a', 38564)]

In [40]:
# 가장 많이 등장하는 단어 10,000개만 남기고 삭제
vocab_size = 10000
truncated_vocabulary = [
    word for word, count in vocabulary.most_common()[:vocab_size]]

In [41]:
# 각 단어를 ID로 바꾸는 전처리
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

In [42]:
# 단어 몇 개에 대한 ID
table.lookup(tf.constant([b"This movie was faaaaaantastic".split()]))

<tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[   22,    12,    11, 10053]], dtype=int64)>

In [43]:
# 최종 훈련 세트
def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch

train_set = datasets["train"].repeat().batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)

In [44]:
# 모델을 만들어 훈련
embed_size = 128
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           mask_zero=True, # not shown in the book
                           input_shape=[None]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_set, steps_per_epoch=train_size // 32, epochs=5)

Epoch 1/5
781/781 [==============================] - 94s 109ms/step - loss: 0.5957 - accuracy: 0.6607
Epoch 2/5
781/781 [==============================] - 86s 111ms/step - loss: 0.3701 - accuracy: 0.8398
Epoch 3/5
781/781 [==============================] - 87s 111ms/step - loss: 0.2081 - accuracy: 0.9237
Epoch 4/5
781/781 [==============================] - 88s 113ms/step - loss: 0.1412 - accuracy: 0.9512
Epoch 5/5
781/781 [==============================] - 87s 112ms/step - loss: 0.1072 - accuracy: 0.9602


## 16.2.1 마스킹
원래 데이터 그대로 모델이 패딩 토큰을 무시하도록 학습되어야하므로 `Embedding` 층을 만들 때 `mask_zero=True` 매개변수를 추가한다. <br/>
구체적으로 `Embedding` 층이 `K.not_equal(inputs, 0)`과 같은 마스크 텐서(mask tensor)를 만든다. <br/>
마스킹 층과 마스크 자동 전파는 `Squential` 모델에 가장 잘 맞고 복잡한 모델에서는 잘 작동하지 않는다.

In [45]:
# 함수형 API를 사용하여 직접 마스킹 처리
K = keras.backend
embed_size = 128
inputs = keras.layers.Input(shape=[None])
mask = keras.layers.Lambda(lambda inputs: K.not_equal(inputs, 0))(inputs)
z = keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size)(inputs)
z = keras.layers.GRU(128, return_sequences=True)(z, mask=mask)
z = keras.layers.GRU(128)(z, mask=mask)
outputs = keras.layers.Dense(1, activation="sigmoid")(z)
model = keras.models.Model(inputs=[inputs], outputs=[outputs])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_set, steps_per_epoch=train_size // 32, epochs=5)

Epoch 1/5
781/781 [==============================] - 92s 109ms/step - loss: 0.6093 - accuracy: 0.6406
Epoch 2/5
781/781 [==============================] - 88s 113ms/step - loss: 0.3712 - accuracy: 0.8425
Epoch 3/5
781/781 [==============================] - 85s 109ms/step - loss: 0.1955 - accuracy: 0.9295
Epoch 4/5
781/781 [==============================] - 85s 109ms/step - loss: 0.1213 - accuracy: 0.9577
Epoch 5/5
781/781 [==============================] - 89s 113ms/step - loss: 0.1054 - accuracy: 0.9617


## 16.2.2 사전훈련된 임베딩 재사용하기
모듈(module) : 모델에 추가하기 쉽게 텐서플로 허브(Hub) 프로젝트가 사전훈련시킨 모델 컴포넌트.

In [47]:
# nnlm-en-dim50 문장 임베딩 모듈 버전 1을 감성 분서거 모델에 사용
tf.random.set_seed(42)

import tensorflow_hub as hub

model = keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1", # 문장 인코더(sentence encoder) 다운로드
                   dtype=tf.string, input_shape=[], output_shape=[50]),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam",
              metrics=["accuracy"])

In [48]:
# IMDb 리뷰 데이터셋 다운로드
import tensorflow_datasets as tfds

datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_size = info.splits["train"].num_examples
batch_size = 32
train_set = datasets["train"].repeat().batch(batch_size).prefetch(1)
history = model.fit(train_set, steps_per_epoch=train_size // batch_size, epochs=5)

Epoch 1/5
781/781 [==============================] - 3s 3ms/step - loss: 0.5861 - accuracy: 0.6919
Epoch 2/5
781/781 [==============================] - 2s 3ms/step - loss: 0.5181 - accuracy: 0.7445
Epoch 3/5
781/781 [==============================] - 2s 3ms/step - loss: 0.5122 - accuracy: 0.7494
Epoch 4/5
781/781 [==============================] - 2s 3ms/step - loss: 0.5086 - accuracy: 0.7492
Epoch 5/5
781/781 [==============================] - 2s 3ms/step - loss: 0.5052 - accuracy: 0.7518


# 16.3 신경망 기계 번역을 위한 인코더-디코더 네트워크
- 샘플링 소프트맥스(sampled softmax) : 타깃 단어에 대한 로짓과 타깃이 아닌 단어 중 무작위로 샘플링한 단어의 로짓만 고려하는 것.

In [51]:
!pip install tensorflow-addons

In [52]:
# 기본적인 인코더-디코더 모델
tf.random.set_seed(42)

import tensorflow_addons as tfa

encoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
decoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
sequence_lengths = keras.layers.Input(shape=[], dtype=np.int32)

embeddings = keras.layers.Embedding(vocab_size, embed_size)
encoder_embeddings = embeddings(encoder_inputs)
decoder_embeddings = embeddings(decoder_inputs)

encoder = keras.layers.LSTM(512, return_state=True) # 최종 은닉 상태를 디코더로 보냄.
encoder_outputs, state_h, state_c = encoder(encoder_embeddings)
encoder_state = [state_h, state_c]

sampler = tfa.seq2seq.sampler.TrainingSampler() # 각 스텝에서 디코더에게 이전 스텝의 출력이 무엇인지 알려주는 샘플러.

decoder_cell = keras.layers.LSTMCell(512) # 은닉 상태 단기와 장기 두 개를 반환.
output_layer = keras.layers.Dense(vocab_size)
decoder = tfa.seq2seq.basic_decoder.BasicDecoder(decoder_cell, sampler,
                                                 output_layer=output_layer)
final_outputs, final_state, final_sequence_lengths = decoder(
    decoder_embeddings, initial_state=encoder_state,
    sequence_length=sequence_lengths)#, training=None)
Y_proba = tf.nn.softmax(final_outputs.rnn_output)

model = keras.models.Model(
    inputs=[encoder_inputs, decoder_inputs, sequence_lengths],
    outputs=[Y_proba])

## 16.3.1 양방향 RNN
일반적인 순환 층은 인과적(casual)이다. 즉, 과거와 현재의 입력만 보고 출력을 생성한다. 
- 양방향 순환 층(bidirectional recurrent layer) : 하나는 왼쪽에서 오른쪽으로 단어를 읽고 다른 하나는 오른쪽에서 왼쪽으로 읽는 두 개의 순환 층.

In [53]:
# 양방향 GRU 층
model = keras.models.Sequential([
    keras.layers.GRU(10, return_sequences=True, input_shape=[None, 10]),
    keras.layers.Bidirectional(keras.layers.GRU(10, return_sequences=True))
])

model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_12 (GRU)                 (None, None, 10)          660       
_________________________________________________________________
bidirectional (Bidirectional (None, None, 20)          1320      
Total params: 1,980
Trainable params: 1,980
Non-trainable params: 0
_________________________________________________________________
